In [1]:
# # ================================
# # INSTALL REQUIRED PACKAGES ()
# # ================================

# # Use the working approach from QuantizationTechniques.ipynb
# !pip install keras
# !pip install tensorflow
# !pip install tensorflow-model-optimization
# !pip install kagglehub --quiet

# print("📦 Required packages installed successfully!")


In [2]:
# ================================
# COLAB ENVIRONMENT SETUP
# ================================

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import kagglehub
import glob
from PIL import Image
import sys

print("🔧 Setting up Colab environment...")
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("🚀 Environment setup complete!")

🔧 Setting up Colab environment...
TensorFlow version: 2.19.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
🚀 Environment setup complete!


In [3]:
# ================================
# IMAGENET MINI DATASET SETUP (TRAIN/TEST SPLIT)
# ================================

import kagglehub
import glob
from PIL import Image
import sys

IMG_SIZE = 224
BATCH_SIZE = 32
MAX_TRAIN_SAMPLES = None  # Use all training samples
MAX_TEST_SAMPLES = None   # Use all test samples

print("🔐 Setting up Kaggle authentication for ImageNet Mini dataset...")
if 'kaggle.json' not in os.listdir():
    sys.exit("Add kaggle.json to access the dataset from Kaggle or place a local dataset folder next to the notebook")

print("📁 Downloading ImageNet Mini dataset from Kaggle...")
try:
    # Download ImageNet Mini dataset with train/test split
    path = kagglehub.dataset_download("ifigotin/imagenetmini-1000")
    IMAGENET_PATH = path
    print("✅ Dataset downloaded successfully!")
    print(f"Dataset path: {IMAGENET_PATH}")
except Exception as e:
    print(f"❌ Error downloading dataset: {e}")
    IMAGENET_PATH = None

print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")


🔐 Setting up Kaggle authentication for ImageNet Mini dataset...
📁 Downloading ImageNet Mini dataset from Kaggle...


100%|██████████| 3.92G/3.92G [02:59<00:00, 23.5MB/s]

Extracting files...


✅ Dataset downloaded successfully!
Dataset path: /root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1
Image size: 224x224
Batch size: 32


In [6]:
!ls  /root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1/"/imagenet-mini"

train  val


In [7]:
IMAGENET_PATH = IMAGENET_PATH+"/imagenet-mini"

In [8]:
# ================================
# DATASET LOADING FUNCTIONS
# ================================

def load_imagenet_train_dataset(imagenet_path, img_size=224, batch_size=32, max_samples=None):
    """Load ImageNet Mini training dataset with proper preprocessing."""
    print(f"📁 Loading ImageNet Mini training samples from: {imagenet_path}")

    train_dir = os.path.join(imagenet_path, "train")
    if not os.path.exists(train_dir):
        raise FileNotFoundError(f"❌ Training folder not found at {train_dir}")

    dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels="inferred",
        label_mode="int",
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=True,  # Shuffle training data
        seed=42
    )

    # Get total count before limiting
    total_samples = tf.data.experimental.cardinality(dataset).numpy() * batch_size

    # Optionally cap dataset
    if max_samples:
        dataset = dataset.unbatch().take(max_samples).batch(batch_size)
        total_samples = min(total_samples, max_samples)

    # Preprocess with EfficientNet normalization
    def preprocess(image, label):
        # EfficientNetV2 expects values in [0, 255] actually!
        # preprocess_input will normalize them internally
        image = tf.cast(image, tf.float32)  # Ensure float32
        image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
        return image, label

    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    print(f"✅ Training dataset ready with {total_samples} samples!")
    return dataset

def load_imagenet_test_dataset(imagenet_path, img_size=224, batch_size=32, max_samples=None):
    """Load ImageNet Mini test dataset with proper preprocessing."""
    print(f"📁 Loading ImageNet Mini test samples from: {imagenet_path}")

    test_dir = os.path.join(imagenet_path, "val")
    if not os.path.exists(test_dir):
        raise FileNotFoundError(f"❌ Test folder not found at {test_dir}")

    dataset = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        labels="inferred",
        label_mode="int",
        image_size=(img_size, img_size),
        batch_size=batch_size,
        shuffle=False  # Don't shuffle test data
    )

    # Get total count before limiting
    total_samples = tf.data.experimental.cardinality(dataset).numpy() * batch_size

    # Optionally cap dataset
    if max_samples:
        dataset = dataset.unbatch().take(max_samples).batch(batch_size)
        total_samples = min(total_samples, max_samples)

    # Preprocess with EfficientNet normalization
    def preprocess(image, label):
        # EfficientNetV2 expects values in [0, 255] actually!
        # preprocess_input will normalize them internally
        image = tf.cast(image, tf.float32)  # Ensure float32
        image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
        return image, label

    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    print(f"✅ Test dataset ready with {total_samples} samples!")
    return dataset


In [9]:
# ================================
# LOAD AND VERIFY DATASETS
# ================================

if IMAGENET_PATH:
    print("🔄 Loading ImageNet Mini datasets...")

    # Load training dataset
    print("\n📚 Loading training dataset...")
    train_dataset = load_imagenet_train_dataset(IMAGENET_PATH, IMG_SIZE, BATCH_SIZE, MAX_TRAIN_SAMPLES)

    # Load test dataset
    print("\n🧪 Loading test dataset...")
    test_dataset = load_imagenet_test_dataset(IMAGENET_PATH, IMG_SIZE, BATCH_SIZE, MAX_TEST_SAMPLES)

    if train_dataset and test_dataset:
        try:
            print("\n🧪 Testing training dataset with one batch...")
            sample_batch, sample_labels = next(iter(train_dataset))
            print(f"✅ Training batch shape: {sample_batch.shape}")
            print(f"🧾 Training labels shape: {sample_labels.shape}")
            print(f"🎨 Pixel range: [{sample_batch.numpy().min():.3f}, {sample_batch.numpy().max():.3f}]")

            print("\n🧪 Testing test dataset with one batch...")
            test_batch, test_labels = next(iter(test_dataset))
            print(f"✅ Test batch shape: {test_batch.shape}")
            print(f"🧾 Test labels shape: {test_labels.shape}")

            print("✅ ImageNet Mini datasets are ready for training and evaluation!")
            DATASET_AVAILABLE = True
        except Exception as e:
            print(f"❌ Error testing datasets: {e}")
            DATASET_AVAILABLE = False
    else:
        print("❌ Could not load datasets properly.")
        DATASET_AVAILABLE = False
else:
    print("❌ ImageNet Mini dataset not available")
    DATASET_AVAILABLE = False

🔄 Loading ImageNet Mini datasets...

📚 Loading training dataset...
📁 Loading ImageNet Mini training samples from: /root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1/imagenet-mini
Found 34745 files belonging to 1000 classes.
✅ Training dataset ready with 34752 samples!

🧪 Loading test dataset...
📁 Loading ImageNet Mini test samples from: /root/.cache/kagglehub/datasets/ifigotin/imagenetmini-1000/versions/1/imagenet-mini
Found 3923 files belonging to 1000 classes.
✅ Test dataset ready with 3936 samples!

🧪 Testing training dataset with one batch...
✅ Training batch shape: (32, 224, 224, 3)
🧾 Training labels shape: (32,)
🎨 Pixel range: [0.000, 255.000]

🧪 Testing test dataset with one batch...
✅ Test batch shape: (32, 224, 224, 3)
🧾 Test labels shape: (32,)
✅ ImageNet Mini datasets are ready for training and evaluation!


In [10]:
# ================================
# CUSTOM QUANTIZE CONFIGS FOR UNSUPPORTED LAYERS
# ================================

print("⚙️ Setting up custom QuantizeConfig for unsupported layers...")

from tensorflow_model_optimization.python.core.quantization.keras import quantize_config

# Custom config for Multiply layer (used in SE blocks)
class NoOpQuantizeConfig(quantize_config.QuantizeConfig):
    """Pass-through quantization config for layers that don't need quantization"""

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass

    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return []

    def get_config(self):
        return {}

# Custom config for Add layer (also used in residual connections)
class DefaultQuantizeConfig(quantize_config.QuantizeConfig):
    """Default quantization for simple layers"""

    def get_weights_and_quantizers(self, layer):
        return []

    def get_activations_and_quantizers(self, layer):
        return []

    def set_quantize_weights(self, layer, quantize_weights):
        pass

    def set_quantize_activations(self, layer, quantize_activations):
        pass

    def get_output_quantizers(self, layer):
        return []

    def get_config(self):
        return {}

print("✅ Custom QuantizeConfig classes defined!")

⚙️ Setting up custom QuantizeConfig for unsupported layers...
✅ Custom QuantizeConfig classes defined!


In [11]:
# # ================================
# # MODEL CREATION
# # ================================

print("🤖 Creating EfficientNetV2B0 model...")

# Load base model without preprocessing layers to avoid QAT compatibility issues
base_model = tf.keras.applications.EfficientNetV2B0(
    include_top=True,
    weights='imagenet',
    input_shape=(224, 224, 3),
    classes=1000,
    include_preprocessing=False  # Critical: avoid Rescaling layer
)

print("✅ Model created successfully!")
print(f"Model parameters: {base_model.count_params():,}")

# Use the model directly
model = base_model

🤖 Creating EfficientNetV2B0 model...
29403144/29403144 [==============================] - 3s 0us/step
✅ Model created successfully!
Model parameters: 7,200,312


In [12]:
# ================================
# CUSTOM QUANTIZATION ANNOTATION
# ================================

print("🔧 Annotating model with custom quantization configs...")

from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer, quantize_annotate_model

def apply_quantization_to_layer(layer):
    """Apply appropriate quantization config based on layer type"""

    # Layers that don't need quantization
    if isinstance(layer, (tf.keras.layers.Multiply,
                         tf.keras.layers.Add,
                         tf.keras.layers.Activation,
                         tf.keras.layers.GlobalAveragePooling2D,
                         tf.keras.layers.Dropout)):
        return quantize_annotate_layer(layer, NoOpQuantizeConfig())

    # Default quantization for other layers
    return layer

# Clone model with custom annotations
print("🔄 Cloning model with quantization annotations...")
annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization_to_layer,
)

print("✅ Model annotated successfully!")

🔧 Annotating model with custom quantization configs...
🔄 Cloning model with quantization annotations...
✅ Model annotated successfully!


In [13]:
from tensorflow_model_optimization.quantization.keras import quantize_scope

print("🔢 Applying quantization to annotated model...")

try:
    # ✅ Apply quantization inside the quantize_scope
    with quantize_scope({'NoOpQuantizeConfig': NoOpQuantizeConfig}):
      # Apply quantization
      qat_model = tfmot.quantization.keras.quantize_apply(annotated_model)

    print("✅ QAT model created successfully!")
    print(f"QAT model parameters: {qat_model.count_params():,}")

except Exception as e:
    print(f"❌ Error applying quantization: {e}")
    print("\n💡 Some layers may still be unsupported. Trying alternative approach...")

    # Alternative: Use selective quantization
    print("🔄 Using selective layer quantization...")

    def selective_quantize(layer):
        """Only quantize Conv2D and Dense layers"""
        if isinstance(layer, (tf.keras.layers.Conv2D, tf.keras.layers.Dense)):
            return tfmot.quantization.keras.quantize_annotate_layer(layer)
        return layer

    annotated_model = tf.keras.models.clone_model(
        model,
        clone_function=selective_quantize,
    )

    qat_model = tfmot.quantization.keras.quantize_apply(annotated_model)
    print("✅ Selective QAT model created!")


🔢 Applying quantization to annotated model...


/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tensorflow.python.ops.init_ops_v2.Constant'>, which may lead to improper serialization.
  warnings.warn(


✅ QAT model created successfully!
QAT model parameters: 7,200,415


In [14]:
# ================================
# MODEL COMPILATION
# ================================

print("⚙️ Compiling QAT model...")

qat_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ QAT model compiled!")

⚙️ Compiling QAT model...
✅ QAT model compiled!


In [15]:
# ================================
# QAT TRAINING WITH IMAGENET MINI
# ================================

if DATASET_AVAILABLE:
    print("🚀 Starting QAT training with ImageNet Mini...")

    # Training parameters
    EPOCHS = 5  # Start with fewer epochs for testing
    LEARNING_RATE = 1e-5

    print(f"📊 Training parameters:")
    print(f"   - Epochs: {EPOCHS}")
    print(f"   - Learning rate: {LEARNING_RATE}")
    print(f"   - Batch size: {BATCH_SIZE}")

    # Compile the QAT model
    print("⚙️ Compiling QAT model...")
    qat_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    print("✅ QAT model compiled!")

    # Start training
    print("🎯 Starting QAT training...")
    history = qat_model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=test_dataset,
        verbose=1
    )

    print("✅ QAT training completed!")

    # Save the trained QAT model
    print("💾 Saving trained QAT model...")
    qat_model.save('qat_trained_model', include_optimizer=False)
    print("✅ Trained QAT model saved to 'qat_trained_model'")

else:
    print("❌ Cannot start QAT training - datasets not available")


🚀 Starting QAT training with ImageNet Mini...
📊 Training parameters:
   - Epochs: 5
   - Learning rate: 1e-05
   - Batch size: 32
⚙️ Compiling QAT model...
✅ QAT model compiled!
🎯 Starting QAT training...
Epoch 1/5
1086/1086 [==============================] - 297s 216ms/step - loss: 0.9554 - accuracy: 0.7882 - val_loss: 1.0251 - val_accuracy: 0.7586
Epoch 2/5
1086/1086 [==============================] - 232s 213ms/step - loss: 0.7807 - accuracy: 0.8122 - val_loss: 0.9922 - val_accuracy: 0.7601
Epoch 3/5
1086/1086 [==============================] - 235s 216ms/step - loss: 0.7040 - accuracy: 0.8267 - val_loss: 0.9629 - val_accuracy: 0.7601
Epoch 4/5
1086/1086 [==============================] - 236s 216ms/step - loss: 0.6552 - accuracy: 0.8385 - val_loss: 0.9582 - val_accuracy: 0.7640
Epoch 5/5
1086/1086 [==============================] - 236s 217ms/step - loss: 0.6074 - accuracy: 0.8474 - val_loss: 0.9470 - val_accuracy: 0.7614
✅ QAT training completed!
💾 Saving trained QAT model...
✅ Tr

In [17]:
# ================================
# EVALUATION ON TEST DATASET
# ================================

if DATASET_AVAILABLE:
    print("📊 Evaluating QAT model on test dataset...")

    # Evaluate the trained QAT model
    test_loss, test_accuracy = qat_model.evaluate(test_dataset, verbose=1)

    print(f"✅ QAT Model Performance:")
    print(f"   - Test Loss: {test_loss:.4f}")
    print(f"   - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

    # Compare with baseline model if available
    if 'model' in locals():
        print("\n🔄 Evaluating baseline model for comparison...")

        # Compile the baseline model before evaluating
        print("⚙️ Compiling baseline model...")
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        print("✅ Baseline model compiled!")

        baseline_loss, baseline_accuracy = model.evaluate(test_dataset, verbose=1)

        print(f"📈 Baseline Model Performance:")
        print(f"   - Test Loss: {baseline_loss:.4f}")
        print(f"   - Test Accuracy: {baseline_accuracy:.4f} ({baseline_accuracy*100:.2f}%)")

        # Calculate accuracy drop
        accuracy_drop = baseline_accuracy - test_accuracy
        print(f"\n📉 Accuracy Drop: {accuracy_drop:.4f} ({accuracy_drop*100:.2f}%)")

        if accuracy_drop < 0.05:  # Less than 5% drop
            print("✅ Good quantization! Accuracy drop is minimal.")
        else:
            print("⚠️  Significant accuracy drop. Consider adjusting quantization parameters.")

else:
    print("❌ Cannot evaluate - datasets not available")

📊 Evaluating QAT model on test dataset...
123/123 [==============================] - 13s 107ms/step - loss: 0.9470 - accuracy: 0.7614
✅ QAT Model Performance:
   - Test Loss: 0.9470
   - Test Accuracy: 0.7614 (76.14%)

🔄 Evaluating baseline model for comparison...
⚙️ Compiling baseline model...
✅ Baseline model compiled!
123/123 [==============================] - 16s 98ms/step - loss: 290.0702 - accuracy: 0.0013
📈 Baseline Model Performance:
   - Test Loss: 290.0702
   - Test Accuracy: 0.0013 (0.13%)

📉 Accuracy Drop: -0.7601 (-76.01%)
✅ Good quantization! Accuracy drop is minimal.


In [ ]:


# ================================
# MODEL EXPORT
# ================================

print("💾 Saving QAT model...")

# Save the QAT-trained model in SavedModel format
qat_model.save('qat_saved_model', include_optimizer=False)

print("✅ QAT model saved to 'qat_saved_model'")
print("🎉 Pipeline complete!")

# ================================
# NEXT STEPS
# ================================

print("\n📋 Next Steps:")
print("1. Convert to TFLite: Use TFLiteConverter to create optimized .tflite file")
print("2. Evaluate performance: Compare accuracy and inference speed")
print("3. Deploy: Use the quantized model in your application")

💾 Saving QAT model...
✅ QAT model saved to 'qat_saved_model'
🎉 Pipeline complete!

📋 Next Steps:
1. Convert to TFLite: Use TFLiteConverter to create optimized .tflite file
2. Evaluate performance: Compare accuracy and inference speed
3. Deploy: Use the quantized model in your application
